# 📓 The GenAI Revolution Cookbook

**Title:** AI Image Generation Simplified: Using DALL·E 2 for Stunning Results

**Description:** Unlock the full potential of AI image generation with DALL·E 2. Master API integration, prompt engineering, and output management for stunning results.

**📖 Read the full article:** [AI Image Generation Simplified: Using DALL·E 2 for Stunning Results](https://blog.thegenairevolution.com/article/ai-image-generation-simplified-using-dalle-2-for-stunning-results)

---

*This jupyter notebook contains executable code examples. Run the cells below to try out the code yourself!*



## Introduction
When I first started working with DALL·E 2, I was honestly amazed at how straightforward the API was. Here's the thing - if you're building any kind of GenAI application that needs visual content, DALL·E 2 gives you a really solid foundation without the complexity you might expect from such powerful tech.

This guide walks you through everything practical: setting up the API, generating images, creating variations, and building production-ready code. No fluff, just what you need to actually build something.

## Why Use DALL·E 2?
Let me be direct here - DALL·E 2 hits the sweet spot for most applications:

<ul>
- **Quality that actually delivers**: The images are genuinely good. Not perfect every time, but consistently usable for real products
- **Dead simple API**: RESTful, works with Python, integrates in minutes not days
- **Flexible enough**: Text-to-image, variations, edits - covers most use cases
- **Production-ready**: OpenAI's infrastructure means you're not babysitting servers
</ul>
I've found it particularly useful when you need custom visuals on demand - whether that's for user-facing features or internal tools.

## Core Concepts
Before diving into code, here's what actually matters:

**Prompts**: Your text descriptions. Be specific. "A red car" gives you garbage. "A vintage red Ferrari on a mountain road at sunset, cinematic lighting" gives you gold.

**Image Generation**: You get three size options (256x256, 512x512, or 1024x1024). Bigger costs more. Start small while testing.

**Variations**: Take an existing image and generate similar ones. Super useful when you're close but not quite there.

**Edits (Inpainting)**: Change specific parts of an image using masks. Honestly more finicky than it sounds but powerful when you get it right.

## Setup
First, get the basics installed:

In [ ]:
!pip install openai pillow requests

Then set up your imports and API key:

In [ ]:
import openai
import os
from PIL import Image
import requests
from io import BytesIO

# Set your OpenAI API key
openai.api_key = os.getenv("OPENAI_API_KEY")  # or set directly: "your-api-key-here"

Get your API key from <a href="https://platform.openai.com/api-keys">OpenAI's platform</a>. And please, use environment variables - don't hardcode it.

## Generating Images from Text
Here's the core functionality - turning text into images:

In [ ]:
def generate_image(prompt, size="1024x1024", n=1):
    """
    Generate images using DALL·E 2
    
    Args:
        prompt: Text description of the desired image
        size: Image dimensions (256x256, 512x512, or 1024x1024)
        n: Number of images to generate (1-10)
    """
    response = openai.Image.create(
        prompt=prompt,
        n=n,
        size=size
    )
    return response['data']

# Generate a single image
prompt = "A futuristic city skyline at sunset with flying cars, digital art style"
images = generate_image(prompt, size="1024x1024", n=1)

# The response contains URLs to the generated images
image_url = images[0]['url']
print(f"Generated image URL: {image_url}")

Now download and save it:

In [ ]:
def download_image(url):
    """Download image from URL and return PIL Image object"""
    response = requests.get(url)
    return Image.open(BytesIO(response.content))

# Download and save the image
img = download_image(image_url)
img.save("generated_image.png")
print("Image saved as generated_image.png")

Quick prompt tips that actually work:

<ul>
- Add style descriptors ("oil painting", "3D render", "photograph")
- Specify lighting and mood
- Include composition details
- The more specific, the better
</ul>
## Creating Image Variations
Sometimes you're close but need alternatives. That's where variations come in:

In [ ]:
def create_variation(image_path, n=1, size="1024x1024"):
    """
    Create variations of an existing image
    
    Args:
        image_path: Path to the source image (must be PNG, < 4MB, square)
        n: Number of variations to generate
        size: Output image size
    """
    with open(image_path, "rb") as image_file:
        response = openai.Image.create_variation(
            image=image_file,
            n=n,
            size=size
        )
    return response['data']

# Create variations of the generated image
variations = create_variation("generated_image.png", n=2)

# Download variations
for i, var in enumerate(variations):
    img = download_image(var['url'])
    img.save(f"variation_{i}.png")
    print(f"Saved variation_{i}.png")

Heads up: Your source image must be square, PNG, and under 4MB. I learned this the hard way.

## Editing Images with Inpainting
Want to change just part of an image? Here's how:

In [ ]:
def edit_image(image_path, mask_path, prompt, n=1, size="1024x1024"):
    """
    Edit an image using a mask and text prompt
    
    Args:
        image_path: Path to the original image (PNG, < 4MB, square)
        mask_path: Path to the mask image (PNG, transparent areas indicate edit regions)
        prompt: Description of the desired edit
        n: Number of edited versions to generate
        size: Output image size
    """
    with open(image_path, "rb") as image_file, open(mask_path, "rb") as mask_file:
        response = openai.Image.create_edit(
            image=image_file,
            mask=mask_file,
            prompt=prompt,
            n=n,
            size=size
        )
    return response['data']

# Example: Edit a specific region
# You need to create a mask image where transparent areas indicate what to edit
edited = edit_image(
    "generated_image.png",
    "mask.png",
    "Add a bright full moon in the sky",
    n=1
)

img = download_image(edited[0]['url'])
img.save("edited_image.png")

Actually, let me show you how to create masks programmatically - way easier than using Photoshop:

In [ ]:
from PIL import Image, ImageDraw

def create_circular_mask(image_path, center, radius, output_path):
    """Create a circular mask for inpainting"""
    img = Image.open(image_path).convert("RGBA")
    mask = Image.new("RGBA", img.size, (0, 0, 0, 255))  # Opaque black
    draw = ImageDraw.Draw(mask)
    
    # Draw transparent circle (area to edit)
    left = center[0] - radius
    top = center[1] - radius
    right = center[0] + radius
    bottom = center[1] + radius
    draw.ellipse([left, top, right, bottom], fill=(0, 0, 0, 0))
    
    mask.save(output_path)
    return output_path

# Create a mask for the top-right region
create_circular_mask("generated_image.png", center=(768, 256), radius=200, output_path="mask.png")

## Building a Production-Ready Generator
Here's where things get real. You need error handling, retries, all that good stuff:

In [ ]:
import time
from typing import List, Optional

class DALLEGenerator:
    """Production-ready DALL·E 2 image generator with error handling"""
    
    def __init__(self, api_key: str, max_retries: int = 3):
        openai.api_key = api_key
        self.max_retries = max_retries
    
    def generate(self, prompt: str, size: str = "1024x1024", n: int = 1) -> List[str]:
        """Generate images with retry logic"""
        for attempt in range(self.max_retries):
            try:
                response = openai.Image.create(
                    prompt=prompt,
                    n=n,
                    size=size
                )
                return [img['url'] for img in response['data']]
            
            except openai.error.RateLimitError:
                if attempt < self.max_retries - 1:
                    wait_time = 2 ** attempt  # Exponential backoff
                    print(f"Rate limit hit. Waiting {wait_time}s...")
                    time.sleep(wait_time)
                else:
                    raise
            
            except openai.error.InvalidRequestError as e:
                print(f"Invalid request: {e}")
                raise
            
            except Exception as e:
                print(f"Unexpected error: {e}")
                if attempt < self.max_retries - 1:
                    time.sleep(1)
                else:
                    raise
    
    def download_and_save(self, url: str, filepath: str) -> None:
        """Download image from URL and save to disk"""
        try:
            response = requests.get(url, timeout=30)
            response.raise_for_status()
            
            img = Image.open(BytesIO(response.content))
            img.save(filepath)
            print(f"Saved image to {filepath}")
        
        except Exception as e:
            print(f"Failed to download image: {e}")
            raise

# Usage example
generator = DALLEGenerator(api_key=os.getenv("OPENAI_API_KEY"))

prompts = [
    "A serene mountain landscape with a lake reflection, photorealistic",
    "Abstract geometric patterns in vibrant colors, modern art style",
    "A cozy coffee shop interior with warm lighting, architectural photography"
]

for i, prompt in enumerate(prompts):
    print(f"\nGenerating image {i+1}: {prompt}")
    urls = generator.generate(prompt, size="1024x1024", n=1)
    generator.download_and_save(urls[0], f"output_{i}.png")

## Run and Evaluate
Let's test everything together:

In [ ]:
# Initialize generator
generator = DALLEGenerator(api_key=os.getenv("OPENAI_API_KEY"))

# Test prompt
test_prompt = "A robot reading a book in a library, warm lighting, digital illustration"

# Generate image
print("Generating image...")
image_urls = generator.generate(test_prompt, size="512x512", n=1)

# Download and save
generator.download_and_save(image_urls[0], "test_output.png")

# Create variations
print("\nCreating variations...")
variations = create_variation("test_output.png", n=2, size="512x512")

for i, var in enumerate(variations):
    generator.download_and_save(var['url'], f"test_variation_{i}.png")

print("\nGeneration complete. Check the output files.")

And here's something practical - track your costs:

In [ ]:
# Track generation costs (approximate)
def estimate_cost(size: str, n: int) -> float:
    """Estimate cost for DALL·E 2 generation (as of 2024)"""
    costs = {
        "1024x1024": 0.020,  # $0.020 per image
        "512x512": 0.018,    # $0.018 per image
        "256x256": 0.016     # $0.016 per image
    }
    return costs.get(size, 0.020) * n

# Calculate cost for batch generation
total_cost = sum(estimate_cost("1024x1024", 1) for _ in prompts)
print(f"Estimated cost for batch: ${total_cost:.3f}")

## Conclusion
That's it - you've got everything you need to integrate DALL·E 2 into your applications. The key things to remember:

<ul>
- Prompts make or break your results. Be specific.
- Always add retry logic - the API will occasionally hiccup
- Start with smaller images while testing to save money
- Variations are your friend when you're almost there
</ul>
What's next? Honestly, just start building. Hook it up to a web app, combine it with GPT for automated prompts, whatever your use case needs. The API is stable enough that you can ship to production without losing sleep.

For the latest updates and detailed docs, check the <a href="https://platform.openai.com/docs/guides/images">OpenAI DALL·E API reference</a>.